###### December 2019 - Roger Melko

Diagonalizing the Hamiltonian matrix for the transverse field Ising model to find the energy eigenvalues and eigenkets.   Calculate the groundstate magnetization.

We will use the same Hamiltonian convention as the QMC program:
$$
H = -J\sum_{\langle i j \rangle} \sigma^z_i \sigma^z_j - B \sum_i \sigma^x_i - h \sum_i \sigma^z_i
$$
where ${\bf \sigma}_i$ are Pauli operators.  In this convention, the 1+1 CFT is at $B/J = 1$ and $h=0$.

In [1]:
using LinearAlgebra

In [27]:
N = 6
Dim = 2^N

J = 1. #exchange interaction
B = 1. #transverse field
h = 0.5 #longitudinal field

Hamiltonian = zeros(Dim,Dim)   #This is your 2D Hamiltonian matrix

Spin2 = 0 #give it a scope outside of the loop

for Ket = 0:Dim-1  #Loop over Hilbert Space
    Diagonal = 0.
    for SpinIndex = 0:N-2  #Loop over spin index (base zero, stop one spin before the end of the chain)
        Spin1 = 2*((Ket>>SpinIndex)&1) - 1
        NextIndex = SpinIndex + 1
        Spin2 = 2*((Ket>>NextIndex)&1) - 1
        Diagonal = Diagonal - J*Spin1*Spin2 - h*Spin1 #spins are +1 and -1
    end
    Diagonal = Diagonal - h*Spin2 #this is the spin at the end of the chain
    
    Hamiltonian[Ket+1,Ket+1] = Diagonal
    
    for SpinIndex = 0:N-1
        bit = 2^SpinIndex   #The "label" of the bit to be flipped
        Bra = Ket ⊻ bit    #Binary XOR flips the bit
        Hamiltonian[Bra+1,Ket+1] = -B
    end
end
Hamiltonian = Hermitian(Hamiltonian);

In the Julia LinearAlgebra package, the eigen function finds eigenvalues and eigenvectors.  They are ordered; i.e. the groundstate energy corresponds to index 1

In [28]:
Diag = eigen(Hamiltonian);

In [29]:
GroundState = Diag.vectors[:, 1];  #this gives the groundstate eigenvector
Diag.values[1] / N

-1.576425410085891

In [30]:
##### Calculate the groundstate magnetization <m^2> in the Z direction
magnetization = zeros(Dim)
abs_mag = zeros(Dim)
mag_squared = zeros(Dim)

SumSz = dropdims(sum(@. (2 * (((0:Dim-1) >> (0:N-1)') & 1) - 1); dims=2); dims=2)
AbsSumSz = abs.(SumSz)
SumSzSq = abs2.(SumSz)

magnetization = SumSz' * abs2.(Diag.vectors)
abs_mag = AbsSumSz' * abs2.(Diag.vectors)
mag_squared = SumSzSq' * abs2.(Diag.vectors)

(magnetization[1] / N), (abs_mag[1] / N), (mag_squared[1] / (N*N))

(0.8651647546885052, 0.8663490216667108, 0.7971193801058516)

In [6]:
beta_vals = [20,15,10,5,3,2,1,0.8,0.5,0.2]
ED =zeros(Float64,length(beta_vals))
idx = 1
for β in beta_vals
    weights = exp.(-β * Diag.values)
    Z = sum(weights)
    E = dot(Diag.values, weights) / (N*Z)
    C = (β^2 * ((dot(Diag.values .^2, weights) / Z) - (N*E)^2))
    # magnetization of thermal state
    M = dot(weights, magnetization) / (N*Z)
    M_abs = dot(weights, abs_mag) / (N*Z)
    M2 = dot(weights, mag_squared) / (N*N*Z)
    println(β," ",E," ",C," ",M," ",M_abs," ",M2)
    #ED[idx,1] = β
    ED[idx,1] = M2
    idx += 1
end

20.0 0.0 0.0 0.0 0.3125 0.16666666666666666
15.0 0.0 0.0 0.0 0.3125 0.16666666666666666
10.0 0.0 0.0 0.0 0.3125 0.16666666666666666
5.0 0.0 0.0 0.0 0.3125 0.16666666666666666
3.0 0.0 0.0 0.0 0.3125 0.16666666666666666
2.0 0.0 0.0 0.0 0.3125 0.16666666666666666
1.0 0.0 0.0 0.0 0.3125 0.16666666666666666
0.8 0.0 0.0 0.0 0.3125 0.16666666666666666
0.5 0.0 0.0 0.0 0.3125 0.16666666666666666
0.2 0.0 0.0 0.0 0.3125 0.16666666666666666
